In [ ]:
from dotenv import load_dotenv
from snowflake.snowpark import Session

load_dotenv()

# Create Snowpark Session
snowflake_connection_parameters = {
    "account": "<ACCOUNT>",
    "user": "<USER>",
    "authenticator": "externalbrowser",
}
snowpark_session = Session.builder.configs(
    snowflake_connection_parameters
).create()

In [ ]:
from trulens.providers.cortex import Cortex

cortex_provider = Cortex(
    model_engine="claude-4-sonnet", snowpark_session=snowpark_session
)

In [ ]:
# Create feedback functions

cortex_feedback_functions = {
    "Logical Consistency": cortex_provider.logical_consistency_with_cot_reasons,
    "Execution Efficiency": cortex_provider.execution_efficiency_with_cot_reasons,
    "Plan Adherence": cortex_provider.plan_adherence_with_cot_reasons,
    "Plan Quality": cortex_provider.plan_quality_with_cot_reasons,
}

for name in cortex_feedback_functions:
    print(f"- {name}")

In [ ]:
trace = """1. User asked what are the three primary colors, 2. Agent first searched up colors of the rainbow, 3. Agent then searched up the three primary colors, 4. Agent then returned the three primary colors"""

## Cortex Provider (Claude-4-sonnet)

In [ ]:
# Out of box execution efficiency feedback
execution_efficiency_feedback = (
    cortex_provider.execution_efficiency_with_cot_reasons(trace)
)
print(execution_efficiency_feedback)

In [ ]:
# Out of box execution efficiency feedback with additional custom instruction
custom_execution_trace_feedback = (
    cortex_provider.execution_efficiency_with_cot_reasons(
        trace, custom_instructions="Ignore the second step in the trace."
    )
)
print(custom_execution_trace_feedback)

In [ ]:
# Override execution efficiency feedback criteria and add additional custom instruction
criteria_override_execution_efficiency_feedback = (
    cortex_provider.execution_efficiency_with_cot_reasons(
        trace,
        criteria="Judge how detailed the trace is.",
        custom_instructions="Ignore the second step in the trace.",
    )
)
print(criteria_override_execution_efficiency_feedback)